# 01 Data Preprocessing
This notebook loads the Olist dataset and performs data merging and feature engineering.

### Collect Data
Download Dataset from Kaggle, set path, and load CSV file

In [7]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

# print("Path to dataset files:", path)

try:
    import kagglehub
except ImportError:
    raise ImportError("kagglehub not installed. Run: pip install kagglehub")

from pathlib import Path
import shutil
import kagglehub
import os

ROOT = Path.cwd().parent
RAW_DIR = ROOT / "data" / "raw" / "olist"
PROCESSED_DIR = ROOT / "data" / "processed"

RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

if len(list(RAW_DIR.glob("*.csv"))) == 0:
    cache_path = Path(kagglehub.dataset_download("olistbr/brazilian-ecommerce"))
    print("Downloaded to cache:", cache_path)

    for f in cache_path.glob("*.csv"):
        shutil.copy2(f, RAW_DIR / f.name)

dataset_path = str(RAW_DIR)
print("Using dataset_path:", dataset_path)
print("Files:", os.listdir(dataset_path))

DEPRECATION: Loading egg at /Users/yoonji/anaconda3/lib/python3.12/site-packages/huggingface_hub-0.29.2-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Downloaded to cache: /Users/yoonji/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/2
Using dataset_path: /Users/yoonji/Projects/personal/olist-return-analysis/data/raw/olist
Files: ['olist_sellers_dataset.csv', 'product_category_name_translation.csv', 'olist_orders_dataset.csv', 'olist_order_items_dataset.csv', 'olist_customers_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_order_payments_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_products_dataset.csv']


In [8]:
# import os

# dataset_path = "/Users/yoonji/.cache/kagglehub/datasets/olistbr/brazilian-ecommerce/versions/2"
# print(os.listdir(dataset_path))  # Check files in the folder

import os
print(os.getcwd())

/Users/yoonji/Projects/personal/olist-return-analysis/notebooks


In [3]:
import pandas as pd

# Set file paths
orders_path = dataset_path + "/olist_orders_dataset.csv"
reviews_path = dataset_path + "/olist_order_reviews_dataset.csv"
products_path = dataset_path + "/olist_products_dataset.csv"
order_items_path = dataset_path + "/olist_order_items_dataset.csv"
category_translation_path = dataset_path + "/product_category_name_translation.csv"
payments_path = dataset_path + "/olist_order_payments_dataset.csv"

# Load data
orders = pd.read_csv(orders_path)
reviews = pd.read_csv(reviews_path)
products = pd.read_csv(products_path)
order_items = pd.read_csv(order_items_path)
category_translation = pd.read_csv(category_translation_path)
payments = pd.read_csv(payments_path)


# Check data
print("Order Data:\n", orders.head())
print("\nReview Data:\n", reviews.head())
print("\nProduct Data:\n", products.head())
print("\nOrder Item Data:\n", order_items.head())
print("\nCategory Name Data:\n", category_translation.head())
print("\nPayment Data:\n", payments.head())

Order Data:
                            order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00     

### Data Preprocessing
Merge review scores and return status into one table

In [4]:
# Select necessary columns
orders_short = orders[['order_id', 'customer_id', 'order_status']]
reviews_short = reviews[['order_id', 'review_score']]
order_items_short = order_items[['order_id', 'product_id']]
products_short = products[['product_id', 'product_category_name']]

# Merge order and review data
df = pd.merge(orders_short, reviews_short, on="order_id")

# Create return status column (1 if canceled, else 0)
df['return_status'] = df['order_status'].apply(lambda x: 1 if x == 'canceled' else 0)

# Merge order item and product data
df = pd.merge(df, order_items_short, on="order_id")
df = pd.merge(df, products_short, on="product_id")

# Check data
print(df.head())

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status  review_score  return_status                        product_id  \
0    delivered             4              0  87285b34884572647811a353c7ac498a   
1    delivered             4              0  595fac2a385ac33a80bd5114aec74eb8   
2    delivered             5              0  aa4383b373c6aca5d8797843e5594415   
3    delivered             5              0  d0b61bfb1de832b15ba9d266ca96e5b0   
4    delivered             5              0  65266b2da20d04dbe00c5c2d3bb7859e   

   product_category_name  
0  utilidades_domesticas  
1             perfum

#### Explanation<br>  
Additional Analysis Required: Only return_status = 0 (not returned) rows are shown at the top, so we should check if returned orders exist(return_status == 1).

In [ ]:
try:
    df.to_parquet(PROCESSED_DIR / "df.parquet", index=False)
    print("Saved:", PROCESSED_DIR / "df.parquet")
except Exception as e:
    print("Parquet save failed -> saving CSV instead.", e)
    df.to_csv(PROCESSED_DIR / "df.csv", index=False)
    print("Saved:", PROCESSED_DIR / "df.csv")